In [1]:
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime as dt                       #import libraries
import csv

In [2]:
def read_csv(fileName):
    data  = []
    input_file = csv.DictReader(open(fileName))                # create function to read data
    for each in input_file:
        data.append(each)
    return data

                                                    
def machine_data(n,cRows,machine_list):
    chosen_data = []
                                                                # segregate dat based on machine id
    for each in cRows:
        if each["machine"] == machine_list[n]:
            chosen_data.append(each)
    return chosen_data
    
                      #WRITE THE CODE TO READ CSV FILE
    
def publish_message(producer_instance, topic_name, data):                                
    try:
        producer_instance.send(topic_name, data)
        #print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
   
    topic = 'process'        #  give topic name
    cRows = read_csv('Streaming_Linux_process.csv')           # read data from function above
    machine_list = []                  # initialize empty list
    
    for i in cRows:
        
                                                                   ## convert to respective data type on producer side
        i['sequence'] = int(i['sequence'])
        i['machine'] = int(i['machine'])
        i['PID'] = int(i['PID'])
        i['TRUN'] = int(i['TRUN'])
        i['TSLPI'] = int(i['TSLPI'])
        i['TSLPU'] = int(i['TSLPU'])
        i['NICE'] = int(i['NICE'])
        i['PRI'] = int(i['PRI'])
        i['RTPR'] = int(i['RTPR'])
        i['CPUNR'] = int(i['CPUNR'])
        i['EXC'] = int(i['EXC'])
        i['CPU'] = float(i['CPU'])
        
        if i['machine'] in machine_list:         # find the no of machine and sort them 
            continue
        else:
            machine_list.append(i['machine'])
    machine_list.sort() 
    
           

    
    print('Publishing records..')
    producer = connect_kafka_producer()
    all_to_send = []                                        # create a list to append list of eaach machine recod based on machine type 
    n=0
    length_chosen_data_list = []
    temp = []                                        # create temporary list in order to append for exhaustion of records
    while n <= (len(machine_list)-1):
        chosen_data = machine_data(n,cRows,machine_list)
        all_to_send.append(chosen_data)
        temp.append(chosen_data)
        length_chosen_data_list.append(len(chosen_data))
        n=n+1
    
    
    while True:                                          # continous streaming 
        for i in range(len(all_to_send)):
            start_index = 0
            no_of_rows = random.randint(10,50)
            print(no_of_rows)                                         #  get random no of rows in range of 
            if len(all_to_send[i]) == 0:
                all_to_send[i] = temp[i]
            each_machine_send = all_to_send[i][:no_of_rows]               # conditions for exhaustion  and replace with origianl data to restart sequence
            rem = no_of_rows - len(each_machine_send)
            if rem > 0:
                all_to_send[i] = temp[i]
                each_machine_send = all_to_send[i][:rem]
            all_to_send[i] = list(filter(lambda x: x not in each_machine_send ,all_to_send[i]))   # filter list so that sequences are not repeated 
            while start_index <= (len(each_machine_send)-1):
                to_send = each_machine_send[start_index]                         
                start_index = start_index+1                     # increment index 
                ts = {'ts': int(dt.datetime.now().timestamp())}
                data=dict(to_send,**ts)               # add time stamp
                print(data)
                publish_message(producer, topic, data)       # publish data 
            
        sleep(5)                # end of cycle 
   
        
        
        
    
            


Publishing records..
15
{'sequence': 1, 'machine': 4, 'PID': 1442, 'TRUN': 0, 'TSLPI': 1, 'TSLPU': 0, 'POLI': 'norm', 'NICE': 0, 'PRI': 120, 'RTPR': 0, 'CPUNR': 0, 'Status': '-', 'EXC': 0, 'State': 'S', 'CPU': 0.0, 'CMD': 'Xorg', 'ts': 1616378260}
{'sequence': 2, 'machine': 4, 'PID': 2797, 'TRUN': 0, 'TSLPI': 5, 'TSLPU': 0, 'POLI': 'norm', 'NICE': 0, 'PRI': 120, 'RTPR': 0, 'CPUNR': 1, 'Status': '-', 'EXC': 0, 'State': 'S', 'CPU': 0.0, 'CMD': 'nautilus', 'ts': 1616378260}
{'sequence': 3, 'machine': 4, 'PID': 2547, 'TRUN': 0, 'TSLPI': 3, 'TSLPU': 0, 'POLI': 'norm', 'NICE': 0, 'PRI': 120, 'RTPR': 0, 'CPUNR': 3, 'Status': '-', 'EXC': 0, 'State': 'S', 'CPU': 0.0, 'CMD': 'unity-panel-se', 'ts': 1616378260}
{'sequence': 4, 'machine': 4, 'PID': 2993, 'TRUN': 0, 'TSLPI': 3, 'TSLPU': 0, 'POLI': 'norm', 'NICE': 0, 'PRI': 120, 'RTPR': 0, 'CPUNR': 3, 'Status': '-', 'EXC': 0, 'State': 'S', 'CPU': 0.0, 'CMD': 'zeitgeist-fts', 'ts': 1616378260}
{'sequence': 5, 'machine': 4, 'PID': 1851, 'TRUN': 0, 'TS

KeyboardInterrupt: 

In [ ]:
print(length_chosen_data_list)

In [ ]:
print(len(all_to_send))